## 常识性预测：
1. 最简单的方式：在不考虑weather poi 等其他一切因素的影响，单单把工作日和节假日区别开来。来进行第一次初步预测。看看结果
2. 加入天气因素

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

### 最简预测：
- 不考虑天气等等因素影响。
- 单单做工作日和非工作日区别，用单日代表法来预测。
- 使用 2016-01-15 作为工作日的数据代表 df1
- 使用 2016-01-16 作为非工作日的数据代表 df0
- 结合数据df10 做一个 数据查询表(ref_dict)
- 在 所要预测的 2016-01-22 到 2016-01-28 中，只有 23，24日是非工作日,因此建立对应关系。
- 得到 最简预测值。

In [5]:
for i in [15,16]:
    if i ==15:
        df1 = pd.read_csv('./order_data_2016-01-{}_gap.csv'.format(i),delimiter=',')
        df1["is_workd"] = np.ones_like(df1.index)
    if i ==16:
        df0 = pd.read_csv('./order_data_2016-01-{}_gap.csv'.format(i),delimiter=',')
        df0["is_workd"] = np.zeros_like(df0.index)
df10 = pd.concat([df1,df0]);df10.head()
#     upload_data = pd.read_csv('season_1/test_0.csv',header=None)
    

,district_id,time_id,gap,is_workd
0,1,1,4,1
1,1,2,7,1
2,1,3,7,1
3,1,4,0,1
4,1,5,1,1


In [15]:
df_test = df10.copy()
# 加 uid 一栏，格式为  “1-1-1” 
df_test["uid"] = df_test.district_id.astype(np.str) + "-" + df_test.time_id.astype(np.str) + "-" + df_test.is_workd.astype(np.str)


In [24]:
# 构建一个 类字典，可以用来 加入预测值
df_ref = df_test.set_index("uid")
df_ref = df_ref.drop(["district_id","time_id","is_workd"],axis=1);df_ref.head()

,gap
uid,
1-1-1,4
1-2-1,7
1-3-1,7
1-4-1,0
1-5-1,1


In [119]:
# 测试发现一些值是缺失的
a = df_ref.ix[df_ref.index == '10-142-0','gap'];a

Series([], Name: gap, dtype: int64)

In [120]:
#构造字典
# a = df_ref.ix[df_ref.index == '1-1-1','gap'].values;int(a)
ref_dict = dict(list(zip(df_ref.index, df_ref.gap)))


#### 构造目标数据集

In [46]:
test = pd.read_csv('season_1/test_0.csv',header=None)

In [47]:
test["day"] = test[1].apply(lambda x: x.split("-")[2])
test["is_workd"] = test["day"].apply(lambda x: 0 if x in ["23","24"] else 1)
test["time_id"] = test[1].apply(lambda x: x.split("-")[-1])
test = test.drop("day", axis =1)
test.columns =["district_id","time_stamp","gap","is_workd","time_id"]
test = test.drop("gap",axis =1)
test.head()

,district_id,time_stamp,is_workd,time_id
0,1,2016-01-22-46,1,46
1,2,2016-01-22-46,1,46
2,3,2016-01-22-46,1,46
3,4,2016-01-22-46,1,46
4,5,2016-01-22-46,1,46


In [49]:
test = test.sort(['is_workd', 'district_id',"time_id"], ascending=[1,1,1]);
test["uid"] =  test.district_id.astype(np.str) + "-" + test.time_id.astype(np.str) + "-" + test.is_workd.astype(np.str)
test.head()

/Users/whale/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,district_id,time_stamp,is_workd,time_id,uid
858,1,2016-01-24-106,0,106,1-106-0
924,1,2016-01-24-118,0,118,1-118-0
990,1,2016-01-24-130,0,130,1-130-0
1056,1,2016-01-24-142,0,142,1-142-0
594,1,2016-01-24-58,0,58,1-58-0


In [113]:
# 有一些值在ref_dict中是缺失的，

no_ref_list = [ uid for uid in test.uid if uid not in df_ref.index]
# 比例为7.8%: 暂时用 0替代
len(no_ref_list)/len(test.index)

0.07787174066243834

In [112]:
test["map_gap"] = np.zeros_like(test.index)
test["map_gap"] = test.uid.apply(lambda x: ref_dict[x] if x not in no_ref_list else 0);test.head()

,district_id,time_stamp,is_workd,time_id,uid,map_gap
858,1,2016-01-24-106,0,106,1-106-0,8
924,1,2016-01-24-118,0,118,1-118-0,5
990,1,2016-01-24-130,0,130,1-130-0,6
1056,1,2016-01-24-142,0,142,1-142-0,2
594,1,2016-01-24-58,0,58,1-58-0,4


In [117]:
test_output = test.copy()
test_output = test_output.drop(['is_workd','time_id','uid'],axis =1)
test_output['map_gap'] = test_output.map_gap.astype('double');test_output.head()

,district_id,time_stamp,map_gap
858,1,2016-01-24-106,8.0
924,1,2016-01-24-118,5.0
990,1,2016-01-24-130,6.0
1056,1,2016-01-24-142,2.0
594,1,2016-01-24-58,4.0


In [118]:
# 输出文件
test_output.to_csv('test_upload_20160531.csv',header=False,index=False)

In [ ]:
# result = pd.merge(test, df10, how='inner', on=['district_id', 'time_id','is_workd'])